<a href="https://colab.research.google.com/github/tul17ii/FP-Tree-With-Patterns/blob/main/FP_Tree_With_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving transactions.csv to transactions (2).csv


In [ ]:
import pandas as pd
from collections import defaultdict

# Load dataset
df = pd.read_csv('transactions.csv')

# Convert the 'Items' column into list format
transactions = df['Items'].apply(lambda x: x.split())
transactions = transactions.tolist()

# Show the transactions
for i, t in enumerate(transactions):
    print(f"T{i+1}: {t}")


T1: ['I1', 'I2', 'I5']
T2: ['I2', 'I4']
T3: ['I2', 'I3']
T4: ['I1', 'I2', 'I4']
T5: ['I1', 'I3']
T6: ['I2', 'I3']
T7: ['I1', 'I3']
T8: ['I1', 'I2', 'I3', 'I5']
T9: ['I1', 'I2', 'I3']


In [ ]:
# Count item frequency
item_counts = defaultdict(int)
for transaction in transactions:
    for item in transaction:
        item_counts[item] += 1

# Set minimum support count (e.g., support = 2, meaning at least 2 transactions)
min_support_count = 2

# Filter out items that do not meet the minimum support count
frequent_items = {item: count for item, count in item_counts.items() if count >= min_support_count}

# Show frequent items
print("Frequent items (>=2 occurrences):", frequent_items)


Frequent items (>=2 occurrences): {'I1': 6, 'I2': 7, 'I5': 2, 'I4': 2, 'I3': 6}


In [ ]:
# Sort frequent items in descending order
frequent_items = dict(sorted(frequent_items.items(), key=lambda x: (-x[1], x[0])))

# Reorder transactions by frequent items
ordered_transactions = []
for transaction in transactions:
    ordered = [item for item in frequent_items if item in transaction]
    ordered_transactions.append(ordered)

# Show the ordered transactions
for i, t in enumerate(ordered_transactions):
    print(f"Ordered T{i+1}: {t}")


Ordered T1: ['I2', 'I1', 'I5']
Ordered T2: ['I2', 'I4']
Ordered T3: ['I2', 'I3']
Ordered T4: ['I2', 'I1', 'I4']
Ordered T5: ['I1', 'I3']
Ordered T6: ['I2', 'I3']
Ordered T7: ['I1', 'I3']
Ordered T8: ['I2', 'I1', 'I3', 'I5']
Ordered T9: ['I2', 'I1', 'I3']


In [ ]:
def find_prefix_paths(base_item, node):
    paths = []
    while node is not None:
        path = []
        parent = node.parent
        while parent is not None and parent.item is not None:
            path.append(parent.item)
            parent = parent.parent
        if path:
            paths.append((path[::-1], node.count))  # Reverse to maintain order from root to leaf
        node = node.link
    return paths


In [ ]:
def build_conditional_tree(paths, min_support_count):
    conditional_transactions = []

    for path, count in paths:
        transaction = []
        for item in path:
            transaction.extend([item] * count)
        conditional_transactions.append(path * count)

    return build_fp_tree(conditional_transactions, min_support_count)


In [ ]:
# FPNode class
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

    def increment(self, count):
        self.count += count

# FP-Tree construction
def build_fp_tree(transactions, min_support_count):
    item_counts = defaultdict(int)

    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1

    frequent_items = {item: count for item, count in item_counts.items() if count >= min_support_count}
    frequent_items = dict(sorted(frequent_items.items(), key=lambda x: (-x[1], x[0])))

    root = FPNode(None, 1, None)
    header_table = {}

    for transaction in transactions:
        filtered_transaction = [item for item in transaction if item in frequent_items]
        filtered_transaction.sort(key=lambda x: frequent_items[x], reverse=True)

        current_node = root
        for item in filtered_transaction:
            if item not in current_node.children:
                new_node = FPNode(item, 1, current_node)
                current_node.children[item] = new_node

                # Header table linking
                if item not in header_table:
                    header_table[item] = new_node
                else:
                    link_node = header_table[item]
                    while link_node.link is not None:
                        link_node = link_node.link
                    link_node.link = new_node
            else:
                current_node.children[item].increment(1)

            current_node = current_node.children[item]

    return root, header_table



In [ ]:
# Build the FP-Tree using ordered transactions
root, header_table = build_fp_tree(ordered_transactions, min_support_count)

frequent_patterns = {}
mine_fp_tree(header_table, [], frequent_patterns, min_support_count)

print("\nFrequent Patterns:")
for pattern, support in frequent_patterns.items():
    print(f'{pattern} -> {support}')



Frequent Patterns:
('I5',) -> 2
('I5', 'I2') -> 2
('I5', 'I1') -> 2
('I5', 'I1', 'I2') -> 2
('I4',) -> 2
('I4', 'I2') -> 2
('I3',) -> 6
('I3', 'I1') -> 4
('I3', 'I1', 'I1') -> 2
('I3', 'I1', 'I1', 'I2') -> 2
('I3', 'I1', 'I2') -> 3
('I3', 'I2') -> 4
('I3', 'I2', 'I2') -> 2
('I1',) -> 6
('I1', 'I2') -> 4
('I1', 'I2', 'I2') -> 6
('I1', 'I2', 'I2', 'I2') -> 4
('I1', 'I2', 'I2', 'I2', 'I2') -> 2
('I2',) -> 7


In [ ]:
import pandas as pd
from collections import defaultdict

# Load the dataset (already done)
df = pd.read_csv('transactions.csv')

# Split the items into lists of items for each transaction
transactions = df['Items'].apply(lambda x: x.split()).tolist()

# FPNode class definition
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

    def increment(self, count):
        self.count += count

# Build FP-Tree function
def build_fp_tree(transactions, min_support_count):
    item_counts = defaultdict(int)

    # Count the frequency of each item in the dataset
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1

    # Filter out items that don't meet the minimum support count
    frequent_items = {item: count for item, count in item_counts.items() if count >= min_support_count}

    # Sort items by frequency (descending) and alphabetically
    frequent_items = dict(sorted(frequent_items.items(), key=lambda x: (-x[1], x[0])))

    # Initialize the FP-Tree root and header table
    root = FPNode(None, 1, None)
    header_table = {}

    # Build the FP-tree from the transactions
    for transaction in transactions:
        filtered_transaction = [item for item in transaction if item in frequent_items]
        filtered_transaction.sort(key=lambda x: frequent_items[x], reverse=True)

        current_node = root
        for item in filtered_transaction:
            if item not in current_node.children:
                new_node = FPNode(item, 1, current_node)
                current_node.children[item] = new_node

                if item not in header_table:
                    header_table[item] = new_node
                else:
                    current_node.children[item].link = header_table[item]
                    header_table[item] = new_node
            else:
                current_node.children[item].increment(1)

            current_node = current_node.children[item]

    return root, header_table

# Print FP-Tree structure
def print_fp_tree(node, level=0):
    if node is not None:
        print('  ' * level + f'{node.item}({node.count})')
        for child in node.children.values():
            print_fp_tree(child, level + 1)

# Set minimum support count (e.g., 2 transactions)
min_support_count = 2

# Build the FP-Tree
root, header_table = build_fp_tree(transactions, min_support_count)

# Print the FP-Tree structure
print("FP-Tree structure:")
print_fp_tree(root)

# Print header table (the item and its linked nodes)
print("\nHeader Table:")
for item, node in header_table.items():
    print(f'{item}: {node.item} -> Count: {node.count}')


FP-Tree structure:
None(1)
  I2(7)
    I1(4)
      I5(1)
      I4(1)
      I3(2)
        I5(1)
    I4(1)
    I3(2)
  I1(2)
    I3(2)

Header Table:
I2: I2 -> Count: 7
I1: I1 -> Count: 2
I5: I5 -> Count: 1
I4: I4 -> Count: 1
I3: I3 -> Count: 2


In [ ]:
import pandas as pd
from collections import defaultdict

# Step 1: Preprocess the dataset and create the transactions list
# Here, we will simulate the dataset
data = {
    'TID': ['T100', 'T200', 'T300', 'T400', 'T500'],
    'Items': [
        'I1 I2 I5',
        'I2 I4 I5',
        'I2 I3 I5',
        'I1 I2 I4 I5',
        'I1 I3 I4 I5'
    ]
}

# Convert to DataFrame
df_new = pd.DataFrame(data)

# Split the 'Items' column into lists for each transaction
transactions = df_new['Items'].apply(lambda x: x.split()).tolist()

# Step 2: Count item frequencies to determine frequent items
item_counts = defaultdict(int)
for transaction in transactions:
    for item in transaction:
        item_counts[item] += 1

# Set the minimum support count (adjustable as needed)
min_support_count = 2

# Filter out items that do not meet the minimum support count
frequent_items = {item: count for item, count in item_counts.items() if count >= min_support_count}

# Step 3: Sort items by frequency (descending) and alphabetically (ascending if tie in count)
sorted_frequent_items = dict(sorted(frequent_items.items(), key=lambda x: (-x[1], x[0])))

# Step 4: Build the FP-tree based on frequent items
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

    def increment(self, count):
        self.count += count

def build_fp_tree(transactions, min_support_count):
    # Create a root node of FP-tree
    root = FPNode(None, 1, None)
    header_table = {}

    for transaction in transactions:
        # Filter items that are not frequent
        filtered_transaction = [item for item in transaction if item in frequent_items]
        # Sort the items by frequency
        filtered_transaction.sort(key=lambda x: frequent_items[x], reverse=True)

        # Traverse the FP-tree and insert the items
        current_node = root
        for item in filtered_transaction:
            if item not in current_node.children:
                new_node = FPNode(item, 1, current_node)
                current_node.children[item] = new_node
                # Add the item to the header table
                if item not in header_table:
                    header_table[item] = new_node
                else:
                    current_node.children[item].link = header_table[item]
                    header_table[item] = new_node
            else:
                current_node.children[item].increment(1)

            current_node = current_node.children[item]

    return root, header_table

# Step 5: Build the FP-tree
root, header_table = build_fp_tree(transactions, min_support_count)

# Step 6: Print the FP-tree structure
def print_fp_tree(node, level=0):
    if node is not None:
        print('  ' * level + f'{node.item}({node.count})')
        for child in node.children.values():
            print_fp_tree(child, level + 1)

# Display the FP-tree
print("FP-Tree structure:")
print_fp_tree(root)

# Step 7: Create the conditional pattern base and FP-tree for each item in the header table
def create_conditional_pattern_base(item, header_table):
    conditional_pattern_base = []
    node = header_table.get(item)

    while node is not None:
        pattern = []
        current_node = node.parent
        while current_node is not None and current_node.item is not None:
            pattern.append(current_node.item)
            current_node = current_node.parent
        if pattern:
            conditional_pattern_base.append((pattern, node.count))
        node = node.link
    return conditional_pattern_base

# Step 8: Generate conditional pattern bases and conditional FP-trees for each item
conditional_pattern_bases = {}
for item in sorted_frequent_items.keys():
    conditional_pattern_base = create_conditional_pattern_base(item, header_table)
    conditional_pattern_bases[item] = conditional_pattern_base

# Display the conditional pattern bases
print("\nConditional Pattern Bases:")
for item, patterns in conditional_pattern_bases.items():
    print(f"{item}: {patterns}")

# Step 9: Generate Conditional FP-Trees (simplified for visualization)
def create_conditional_fp_tree(pattern_base):
    tree = defaultdict(int)
    for pattern, count in pattern_base:
        for item in pattern:
            tree[item] += count
    return tree

# Display Conditional FP-Trees for each item
print("\nConditional FP-Trees:")
for item, pattern_base in conditional_pattern_bases.items():
    conditional_fp_tree = create_conditional_fp_tree(pattern_base)
    print(f"Conditional FP-tree for {item}: {conditional_fp_tree}")


FP-Tree structure:
None(1)
  I5(5)
    I2(4)
      I1(2)
        I4(1)
      I4(1)
      I3(1)
    I1(1)
      I4(1)
        I3(1)

Conditional Pattern Bases:
I5: []
I2: [(['I5'], 4)]
I1: [(['I5'], 1), (['I2', 'I5'], 2)]
I4: [(['I1', 'I5'], 1), (['I1', 'I2', 'I5'], 1), (['I2', 'I5'], 1)]
I3: [(['I4', 'I1', 'I5'], 1), (['I2', 'I5'], 1)]

Conditional FP-Trees:
Conditional FP-tree for I5: defaultdict(<class 'int'>, {})
Conditional FP-tree for I2: defaultdict(<class 'int'>, {'I5': 4})
Conditional FP-tree for I1: defaultdict(<class 'int'>, {'I5': 3, 'I2': 2})
Conditional FP-tree for I4: defaultdict(<class 'int'>, {'I1': 2, 'I5': 3, 'I2': 2})
Conditional FP-tree for I3: defaultdict(<class 'int'>, {'I4': 1, 'I1': 1, 'I5': 2, 'I2': 1})


In [ ]:
import pandas as pd
from collections import defaultdict
from google.colab import files

# Upload and read CSV
uploaded = files.upload()
df = pd.read_csv('transactions.csv')

# Convert 'Items' column to transaction list
transactions = df['Items'].apply(lambda x: x.split()).tolist()

# Show transactions
print("Transactions:")
for i, t in enumerate(transactions):
    print(f"T{i+1}: {t}")

# Step 1: Count item frequency
item_counts = defaultdict(int)
for transaction in transactions:
    for item in transaction:
        item_counts[item] += 1

# Minimum support count
min_support_count = 2

# Filter frequent items
frequent_items = {item: count for item, count in item_counts.items() if count >= min_support_count}
frequent_items = dict(sorted(frequent_items.items(), key=lambda x: (-x[1], x[0])))

# Reorder transactions by frequent item order
ordered_transactions = []
for transaction in transactions:
    ordered = [item for item in frequent_items if item in transaction]
    ordered_transactions.append(ordered)

# FPNode class
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

    def increment(self, count):
        self.count += count

# Build FP-Tree
def build_fp_tree(transactions, min_support_count):
    item_counts = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1

    frequent_items = {item: count for item, count in item_counts.items() if count >= min_support_count}
    frequent_items = dict(sorted(frequent_items.items(), key=lambda x: (-x[1], x[0])))

    root = FPNode(None, 1, None)
    header_table = {}

    for transaction in transactions:
        filtered_transaction = [item for item in transaction if item in frequent_items]
        filtered_transaction.sort(key=lambda x: frequent_items[x], reverse=True)

        current_node = root
        for item in filtered_transaction:
            if item not in current_node.children:
                new_node = FPNode(item, 1, current_node)
                current_node.children[item] = new_node

                if item not in header_table:
                    header_table[item] = new_node
                else:
                    link_node = header_table[item]
                    while link_node.link:
                        link_node = link_node.link
                    link_node.link = new_node
            else:
                current_node.children[item].increment(1)

            current_node = current_node.children[item]

    return root, header_table

# Print FP-tree
def print_fp_tree(node, indent=0):
    if node.item is not None:
        print("  " * indent + f"{node.item} ({node.count})")
    for child in node.children.values():
        print_fp_tree(child, indent + 1)

# Find prefix paths
def find_prefix_paths(base_item, node):
    paths = []
    while node:
        path = []
        parent = node.parent
        while parent and parent.item is not None:
            path.append(parent.item)
            parent = parent.parent
        if path:
            paths.append((path[::-1], node.count))
        node = node.link
    return paths

# Build conditional FP-tree
def build_conditional_tree(paths, min_support_count):
    conditional_transactions = []
    for path, count in paths:
        conditional_transactions.extend([path] * count)
    return build_fp_tree(conditional_transactions, min_support_count)

# Simple pattern miner (recursive)
def mine_fp_tree(header_table, prefix, frequent_patterns, min_support_count):
    sorted_items = sorted(header_table.items(), key=lambda x: x[1].count)
    for base_item, node in sorted_items:
        new_pattern = prefix + [base_item]
        support = 0
        temp = node
        while temp:
            support += temp.count
            temp = temp.link
        frequent_patterns[tuple(new_pattern)] = support

        paths = find_prefix_paths(base_item, header_table[base_item])
        conditional_root, conditional_header = build_conditional_tree(paths, min_support_count)

        if conditional_header:
            mine_fp_tree(conditional_header, new_pattern, frequent_patterns, min_support_count)

# Main verbose miner
def mine_fp_tree_verbose(header_table, prefix, frequent_patterns, min_support_count):
    sorted_items = sorted(header_table.items(), key=lambda x: x[1].count)

    print(f"\n{'Item':<10} {'Conditional Pattern Base':<40} {'Conditional FP-tree':<30} {'Frequent Patterns Generated'}")
    print("-" * 110)

    for base_item, node in sorted_items:
        new_pattern = prefix + [base_item]
        support = 0
        temp = node
        while temp:
            support += temp.count
            temp = temp.link
        frequent_patterns[tuple(new_pattern)] = support

        # 1. Conditional pattern base
        paths = find_prefix_paths(base_item, header_table[base_item])
        cond_base_str = "{" + ", ".join(f"[{', '.join(path)}: {count}]" for path, count in paths) + "}"

        # 2. Conditional FP-tree
        conditional_root, conditional_header = build_conditional_tree(paths, min_support_count)
        cond_tree_counts = []
        if conditional_header:
            for item, n in conditional_header.items():
                total = 0
                node_iter = n
                while node_iter:
                    total += node_iter.count
                    node_iter = node_iter.link
                cond_tree_counts.append(f"({item}: {total})")
        cond_tree_str = ", ".join(cond_tree_counts)

        # 3. Frequent patterns from subtree
        sub_patterns = []
        if conditional_header:
            sub_freq = {}
            mine_fp_tree(conditional_header, new_pattern, sub_freq, min_support_count)
            for pattern, sup in sub_freq.items():
                frequent_patterns[pattern] = sup
                sub_patterns.append(f"{', '.join(pattern)}: {sup}")
        else:
            sub_patterns = [f"{', '.join(new_pattern)}: {support}"]

        print(f"{base_item:<10} {cond_base_str:<40} {cond_tree_str:<30} {{{'}}, {{'.join(sub_patterns)}}}")

# === Run FP-Growth ===
root, header_table = build_fp_tree(ordered_transactions, min_support_count)

print("\nFull FP-tree:")
print_fp_tree(root)

frequent_patterns = {}
mine_fp_tree_verbose(header_table, [], frequent_patterns, min_support_count)


Saving transactions.csv to transactions (3).csv
Transactions:
T1: ['I1', 'I2', 'I5']
T2: ['I2', 'I4']
T3: ['I2', 'I3']
T4: ['I1', 'I2', 'I4']
T5: ['I1', 'I3']
T6: ['I2', 'I3']
T7: ['I1', 'I3']
T8: ['I1', 'I2', 'I3', 'I5']
T9: ['I1', 'I2', 'I3']

Full FP-tree:
  I2 (7)
    I1 (4)
      I5 (1)
      I4 (1)
      I3 (2)
        I5 (1)
    I4 (1)
    I3 (2)
  I1 (2)
    I3 (2)

Item       Conditional Pattern Base                 Conditional FP-tree            Frequent Patterns Generated
--------------------------------------------------------------------------------------------------------------
I5         {[I2, I1: 1], [I2, I1, I3: 1]}           (I2: 2), (I1: 2)               {I5, I2: 2}}, {{I5, I1: 2}}, {{I5, I1, I2: 2}
I4         {[I2: 1], [I2, I1: 1]}                   (I2: 2)                        {I4, I2: 2}
I3         {[I2: 2], [I1: 2], [I2, I1: 2]}          (I2: 4), (I1: 4)               {I3, I1: 4}}, {{I3, I1, I2: 2}}, {{I3, I2: 4}
I1         {[I2: 4]}                            